# word2vector 기반 유저 프로파일 / 딜 피쳐
* base_model 변형한 base_model_simple로 시도

## 데이터 생성: MakingTrainData.ipynb

1. 기간: 3월11일~4월11일 위픽 시퀀스
2. 학습 및 밸리데이션에는: 1-19 슬롯 제외. 100 이상도 제외
3. 학습: 3월 21일 ~ 4월 7일
4. 밸리데이션: 4월 8일 ~ 4월 10일
5. 예측: 4월 11일
6. 약 14M rows(학습데이터): train_data_w2vec.csv, 
7. test_data_w2vec.csv

### 생성 디테일
- 유저별 정렬된 위픽 히스토리 생성: 
    `sort -t, -k2,2n -k4n pos_dataset.csv > pos_dataset_by_user.csv`
    
- 각 유저별로 시간순으로 정렬된 시퀀스에 대해
    - (pos, neg) 쌍으로 추출
        - dt 별로 그룹화하여
            - gte_slot (20)이상 ~ 100 미만 슬롯의 클릭을 우선 positive example로 선정
            - 해당 positive example의 슬롯 상위 window 범위 내에서 랜덤 샘플링하여 negative example 짝을 선정
                - 샘플링시, dt 내에서 일으킨 클릭은 제외함
    - (pos, neg)에 대한 유저 프로파일을 정의할 히스토리 정의
        - 히스토리 길이가 min_profile_cnt (4) 이상인 것만 골라서, 최근 기준 max_profile_cnt 길이로 히스토리를 제한
        - 각 유저별로 히스토리의 갯수 (샘플 수)가 max_sample_per_user 넘지 않게 재한
    - 그 후, word2vec 을 조회하여, user_profile, deal_profile 생성
    
## 가공: make_train_data.ipynb

1. pandas로 학습데이터/밸리데이션 데이터를 읽어서 base_model_simple에 사용할 포맷으로 변경
2. 메모리 문제로 인해 train_set은 700K 정도의 rows로 샘플링
3. train_set format: "u", "ts", "v", "slot", "label", user_profile:100, deal_feature:100
4. test_set format: "u", "ts", "p_v", "p_slot", "n_v", "n_slot", user_profile:100, pos_deal_feature:100, neg_dealfeature:100
5. w2v_dataset.pkl

## 예측을 위한 특정일/시간의 위픽세팅 데이터: MakingTrainData.ipynb (하단)

1. 위픽세팅의 20-100등 사이의 딜들을 미리 고정 예측 대상 딜들로 만들어 둠
2. for_predict_w2v_dataset.pkl

In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from operator import itemgetter

# 한글 출력준비

* 네이버 나눔고딕 폰트 등을 설치 또는 사용
* 폰트를 새로 설치 했다면 아래 실행
    ```
    import matplotlib as mpl
    mpl.font_manager._rebuild()
    ```

* 한글 폰트 사용하려면 관련 폰트를 등록하면 된다. (나의 경우는 나눔코딩 폰트)
    ```
    path = 'c:\\windows\\fonts\\nanumgothiccoding.ttf'
    font_name = fm.FontProperties(fname=path, size=50).get_name()
    plt.rc('font', family=font_name)
    ```


In [2]:
path = 'c:\\windows\\fonts\\nanumgothiccoding.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

# 예측 결과 로딩

In [3]:
def load_pred_result(file_path):
    pred_dic = {}
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)        
        for user, dt, v, slot, score in reader:
            if user not in pred_dic:
                pred_dic[user] = []
            pred_dic[user].append((v, slot, score, dt))
    return pred_dic

In [4]:
def load_item_b(file_path):
    item_b= []
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)        
        for row in reader:
            item_b.append(row)
    return item_b

# Elastic Search 를 이용

In [5]:
es = elasticsearch.Elasticsearch('twiceSpark1:9200')

def es_search(es, deal_ids):
    res = es.search(index='ojm5', body={'query': {'ids':{'values':deal_ids}}}, size=100)
    n = res['hits']['total']
    dic = {}
    for hit in res['hits']['hits']:
        dic[hit['_id']] =  hit['_source']
    return dic
    

In [6]:
def show_preds(pred):
    img_base_url=r'http://img.wemep.co.kr/'
    history = es_search(es, pred[0])
        
    fig, axes = plt.subplots(ncols=len(pred[0]), figsize=(16,16))
    
    for i, h in enumerate(pred[0]):
        title = history[h]['_2']
        img = history[h]['_3']
        f = urllib.request.urlopen(img_base_url+img)
        a = plt.imread(f, format='jpg')
        axes[i].imshow(a)
        axes[i].set_title("{}".format(title))
        axes[i].axis('off')
    plt.show()
    
    candidates = list(map(lambda x: x[0], pred[1]))
    res = es_search(es, candidates)
    
    i = 0
    for prod, slot, prob in pred[1]:
        i+=1
        if prod in res:
            cat = res[prod]['_4']['_5']
            title = res[prod]['_2']
            img = res[prod]['_3']
            #print('{}[{}].{}.prod {}.slot {}.prob {}'.format(title, cat, img, prod, slot, prob))
            #continue
            
            f = urllib.request.urlopen(img_base_url+img)        
            # read the image file in a numpy array
            a = plt.imread(f, format='jpg')
            plt.imshow(a)
            plt.title("{}[{}] score:{} {} [{}]".format(i, slot, prob, title, cat))
            plt.axis('off')
            plt.show()

        else:
            print('no entry for product {}.slot {}.prob {}'.format(prod, slot, prob))

In [7]:
def show_history_only(pred):
    img_base_url=r'http://img.wemep.co.kr/'
    history = es_search(es, pred[0])
        
    fig, axes = plt.subplots(ncols=len(pred[0]), figsize=(16,16))
    
    for i, h in enumerate(pred[0]):
        title = history[h]['_2']
        img = history[h]['_3']
        f = urllib.request.urlopen(img_base_url+img)
        a = plt.imread(f, format='jpg')
        axes[i].imshow(a)
        axes[i].set_title("{}".format(title))
        axes[i].axis('off')
    plt.show()

In [8]:
def show_item_b(item_b, show_figure=False):
    img_base_url=r'http://img.wemep.co.kr/'
    
    candidates = list(map(lambda x: x[0], item_b))
    res = es_search(es, candidates)
    
    i = 0
    for deal, slot, score in item_b:
        i+=1
        if deal in res:
            cat = res[deal]['_4']['_5']
            title = res[deal]['_2']
            img = res[deal]['_3']
            
            f = urllib.request.urlopen(img_base_url+img)        
            # read the image file in a numpy array
            a = plt.imread(f, format='jpg')
            plt.imshow(a)
            plt.title("{}[{}] score:{} {} [{}]".format(i, slot, score, title, cat))
            plt.axis('off')
            plt.show()

        else:
            print('no entry for product {} score{}'.format(deal, score))

In [9]:
def es_scan_extra_by_dids(dids):
    """
    dids로부터, mn, tn1를 가져온다.
    """
    res = es.search(index='dealinfos', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["mn", "tn1", "did"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    for hit in res['hits']['hits']:
        dic[hit['_source']['did']] = (hit['_source']['mn'], hit['_source']['tn1'])
    return dic

In [10]:
def es_read_wepick_setting(dt, start_slot=20):
    """
    위픽 세팅 로딩
    """
    res = es.search(index='wepick_setting', 
                body={
                    'query': {
                        'term': {'dt': dt }
                        }                        
                    }
               )
    if res['hits']['total'] > 0:
        dic = {}
        vec = []
        for s in res['hits']['hits'][0]['_source']['setting']:
            row = s.split('_')
            dic[int(row[0])] = int(row[1])
        for slot, did in dic.items():
            if slot >= start_slot:
                vec.append(did)
        return vec, dic
    return None, None

In [132]:
def get_w2v_words_from_did(dids):
    """
    해당 딜번호의 워드 모음
    """
    res = es.search(index='deal_word2vec', 
                body={
                  "query": { "terms": {"v":dids } },
                  "_source": ["words", "v"],
                    "size": len(dids)
                    }
               )
    dic = {}
    if res['hits']['total'] > 0:
        for hits in res['hits']['hits']:
            dic[hits['_source']['v']] = hits['_source']['words']
    return dic

In [154]:
def print_user_wepick_history(user_id, day_limit, gte_slot=20, lte_slot=100):
    """
    사용자의 위픽 히스토리를 출력
    day_limit 미만
    클릭한 슬롯이: gte_slot ~ lte_slot 이어야 한다.
    """
    res = es.search(index='wepick_seq', 
                body={
                  "query": {
                    "bool": {
                      "must": { 
                        "term": {"u":user_id}
                      },
                      "filter": [
                          {
                            "range" : {
                              "rgtime" : {
                              "lt" : day_limit
                              }
                            }                              
                          },
                          {
                            "range" : {
                              "slot" : {
                              "gte" : gte_slot,
                              "lte" : lte_slot
                              }
                            }                              
                          }
                      ]
                    }
                  },
                  "size": 128,
                  "sort": {"rgtime":"desc"}
                                }
               )
    filtered = []
    if res['hits']['total'] > 0:
        filtered = []
        for hit in res['hits']['hits']:
            filtered.append((hit['_source']['v'], hit['_source']['rgtime'], hit['_source']['slot']))
    if len(filtered) > 0:
        ret = []
        deal_dic = es_scan_extra_by_dids(list(map(lambda x: x[0], filtered)))
        deal_dcs = get_w2v_words_from_did(list(map(lambda x: x[0], filtered)))
        for did, ts, slot in filtered:
            if did in deal_dic:
                if did in deal_dcs:
                    dcs = deal_dcs[did]
                else:
                    dcs = []
                ret.append((did, ts, slot, deal_dic[did][0], deal_dic[did][1], dcs))
                #print("{},{},{},{},{}".format(did, ts, slot, deal_dic[did][0], deal_dic[did][1]))
        filtered = ret
    return filtered

In [156]:
result = print_user_wepick_history(1000007, '2018-04-10', gte_slot=1)
for did, ts, slot, title, tn, dcs in result:
    print(did, ts, slot, title, tn)

3523535 2018-04-09T23:10:44.416000 29 [위메프] 10만 포인트+5,000P 온라인 이용권
3503625 2018-04-05T05:41:09.979000 1 [무료배송] 4월의 아디다스 KIDS 유아동신발
3503625 2018-04-05T05:40:51.474000 1 [무료배송] 4월의 아디다스 KIDS 유아동신발
3499380 2018-04-05T05:28:31.716000 4 [플레이특가] 원마운트 워터파크 1+1 워터파크/스파
3499380 2018-04-05T03:00:46.358000 4 [플레이특가] 원마운트 워터파크 1+1 워터파크/스파
3503625 2018-04-05T03:00:07.390000 1 [무료배송] 4월의 아디다스 KIDS 유아동신발
3504535 2018-04-05T02:53:58.618000 6 [투데이특가] 라피타제모기 런칭행사 제모기
3499380 2018-04-05T02:47:47.280000 4 [플레이특가] 원마운트 워터파크 1+1 워터파크/스파
3499380 2018-04-05T02:47:29.358000 4 [플레이특가] 원마운트 워터파크 1+1 워터파크/스파
3503625 2018-04-05T01:57:21.875000 1 [무료배송] 4월의 아디다스 KIDS 유아동신발
3493647 2018-04-04T12:09:39.259000 1 [투데이특가] 챔피온 반팔티셔츠 여성 티셔츠/상의 기타
3476054 2018-04-04T07:28:51.280000 1 [투데이특가] 롯데마트 모바일금액권 생활쿠폰
3493647 2018-04-04T02:24:20.160000 2 [투데이특가] 챔피온 반팔티셔츠 여성 티셔츠/상의 기타
3483941 2018-04-02T11:19:45.623000 1 [디지털데이] 닌텐도 스위치+타이틀 휴대용 게임기
3489914 2018-04-02T05:22:06.758000 6 [무료배송] 리얼코코 기습추가할인 자켓/코트
3480792 2018-04-02T05:1

In [158]:
for did, ts, slot, title, tn, dcs in result:
    print(did, title, dcs)

3523535 [위메프] 10만 포인트+5,000P ['위멥포인트', '선물하기', '연말선물', '포인트', '연말']
3503625 [무료배송] 4월의 아디다스 KIDS ['아디다스', '슈퍼스타', '슈퍼스타', '아동슬리퍼', '아동슈즈']
3503625 [무료배송] 4월의 아디다스 KIDS ['아디다스', '슈퍼스타', '슈퍼스타', '아동슬리퍼', '아동슈즈']
3499380 [플레이특가] 원마운트 워터파크 1+1 ['고양', '가족', '물놀이', '어린이', '여행']
3499380 [플레이특가] 원마운트 워터파크 1+1 ['고양', '가족', '물놀이', '어린이', '여행']
3503625 [무료배송] 4월의 아디다스 KIDS ['아디다스', '슈퍼스타', '슈퍼스타', '아동슬리퍼', '아동슈즈']
3504535 [투데이특가] 라피타제모기 런칭행사 ['제모크림', '레이저제모기', '라피타ipl', '제모기', '홈케어']
3499380 [플레이특가] 원마운트 워터파크 1+1 ['고양', '가족', '물놀이', '어린이', '여행']
3499380 [플레이특가] 원마운트 워터파크 1+1 ['고양', '가족', '물놀이', '어린이', '여행']
3503625 [무료배송] 4월의 아디다스 KIDS ['아디다스', '슈퍼스타', '슈퍼스타', '아동슬리퍼', '아동슈즈']
3493647 [투데이특가] 챔피온 반팔티셔츠 ['챔피온로고', '챔피온반팔', 't425', '반팔티셔츠', '챔피온']
3476054 [투데이특가] 롯데마트 모바일금액권 ['디지털상품권', '모바일금액권', '롯데마트상품권', '롯데상품권', '마트']
3493647 [투데이특가] 챔피온 반팔티셔츠 ['챔피온로고', '챔피온반팔', 't425', '반팔티셔츠', '챔피온']
3483941 [디지털데이] 닌텐도 스위치+타이틀 ['마리오', '후대용게임기', '닌텐도스위치', '젤다의전설', '닌텐도']
3489914 [무료배송] 리얼코코 기습추가할인 ['리얼코코', '치마'

## Ranking 관련

In [11]:
def print_result(predictions, extra):
    for _did, slot, score, _ in predictions:
        did = int(_did)
        if did in extra:
            print((score, did, extra[did][0], slot, extra[did][1]))

In [12]:
def print_predicted_ranks(prediction):
    print_result(prediction, es_scan_extra_by_dids(list(map(lambda x: int(x[0]), prediction))))

In [13]:
def print_item_bias(item_b):
    print_result(item_b, es_scan_extra_by_dids(list(map(lambda x: int(x[0]), item_b))))

In [ ]:
# 2018-04-11 21 시의 위픽 세팅 로딩
wepick_setting, wepick_dic = es_read_wepick_setting('2018-04-11 21')

## 사용자 히스토리 보기

In [99]:
print_user_wepick_history(1000007, '2018-04-11', gte_slot=1)

3513125,2018-04-10T12:35:10.299000,1,[슈퍼투데이특가] 호텔수건 170g10장,수건/타올
3513125,2018-04-10T12:34:47.185000,1,[슈퍼투데이특가] 호텔수건 170g10장,수건/타올
3513125,2018-04-10T12:28:49.038000,1,[슈퍼투데이특가] 호텔수건 170g10장,수건/타올
3513125,2018-04-10T10:29:00.445000,1,[슈퍼투데이특가] 호텔수건 170g10장,수건/타올
3513125,2018-04-10T09:16:01.431000,1,[슈퍼투데이특가] 호텔수건 170g10장,수건/타올
3513125,2018-04-10T09:15:15.176000,1,[슈퍼투데이특가] 호텔수건 170g10장,수건/타올
3513125,2018-04-10T09:14:56.132000,1,[슈퍼투데이특가] 호텔수건 170g10장,수건/타올
3523535,2018-04-09T23:10:44.416000,29,[위메프] 10만 포인트+5,000P,온라인 이용권
3503625,2018-04-05T05:41:09.979000,1,[무료배송] 4월의 아디다스 KIDS,유아동신발
3503625,2018-04-05T05:40:51.474000,1,[무료배송] 4월의 아디다스 KIDS,유아동신발
3499380,2018-04-05T05:28:31.716000,4,[플레이특가] 원마운트 워터파크 1+1,워터파크/스파
3499380,2018-04-05T03:00:46.358000,4,[플레이특가] 원마운트 워터파크 1+1,워터파크/스파
3503625,2018-04-05T03:00:07.390000,1,[무료배송] 4월의 아디다스 KIDS,유아동신발
3504535,2018-04-05T02:53:58.618000,6,[투데이특가] 라피타제모기 런칭행사,제모기
3499380,2018-04-05T02:47:47.280000,4,[플레이특가] 원마운트 워터파크 1+1,워터파크/스파
3499380,2018-04-0

# 슬롯 바이어스 있음

# wepick_pred_a128_32_16_sb

In [30]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a128_32_16_sb.csv')

In [31]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6456570625305176', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6086955070495605', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5995533466339111', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5901321172714233', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.5899836421012878', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5827142596244812', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5724421739578247', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5587748289108276', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5367603898048401', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5325303077697754', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5191582441329956', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5159433484077454', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5110401511192322', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5078080296516418', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.50282877

# wepick_pred_a64_32_16_sb

In [32]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a64_32_16_sb.csv')

In [33]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6456570625305176', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6086955070495605', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5995532274246216', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5901318788528442', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.5899835824966431', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5827140808105469', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5724421143531799', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5587747097015381', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5367603302001953', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5325303077697754', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5191581845283508', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5159434080123901', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5110403299331665', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5078079104423523', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.50282871

# wepick_pred_a32_32_16_sb

In [34]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a32_32_16_sb.csv')

In [35]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6456570625305176', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6086955070495605', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5995532870292664', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5901321172714233', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.5899836421012878', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5827141404151917', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5724421739578247', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5587748289108276', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5367602109909058', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5325303673744202', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5191581845283508', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5159433484077454', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.511040210723877', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5078080296516418', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.502828776

# wepick_pred_a16_32_16_sb

In [36]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a16_32_16_sb.csv')

In [37]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6456570625305176', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6086955070495605', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5995534062385559', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5901321172714233', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.5899837017059326', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5827142596244812', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5724421739578247', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5587747693061829', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5367602705955505', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5325303077697754', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5191582441329956', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5159433484077454', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5110401511192322', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5078080296516418', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.50282877

# wepick_pred_a4_32_16_sb

In [38]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a4_32_16_sb.csv')

In [39]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6456570625305176', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6086955070495605', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5995533466339111', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5901320576667786', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.5899836421012878', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5827141404151917', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5724421143531799', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5587746500968933', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5367600321769714', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5325303673744202', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5191582441329956', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5159433484077454', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5110402703285217', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5078079700469971', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.50282877

# 슬롯 바이어스 없음

# wepick_pred_a128_32_16

In [21]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a128_32_16.csv')

In [22]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6306593418121338', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6161671280860901', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5934129953384399', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5929120182991028', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5914928913116455', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5841131806373596', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.539723813533783', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5395549535751343', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5367130041122437', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5264455080032349', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5223358273506165', 3524294, '[투데이특가] 여심저격 클루나드시계', '83', '패션시계')
('0.5090673565864563', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5037084221839905', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5032361149787903', 3544419, '[게릴라특가] 봄맞이 카페트 150x200', '71', '카페트/러그')
('0.5

# wepick_pred_a64_32_16

In [18]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a64_32_16.csv')

In [19]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6176784038543701', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6173945665359497', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5893151760101318', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5791898369789124', 3524294, '[투데이특가] 여심저격 클루나드시계', '83', '패션시계')
('0.5751028656959534', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.549614429473877', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.5458723902702332', 3515524, '[무료배송] 롱티/티셔츠/원피스', '45', '티셔츠')
('0.5456531047821045', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5431188344955444', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5409591794013977', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5401630997657776', 3513787, '[리빙위크] 3M 정전기청소포/물걸레', '67', '밀대/청소포')
('0.5363984107971191', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5305687189102173', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5231457948684692', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5156151056

## wepick_pred_a32_32_16

In [23]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a32_32_16.csv')

In [24]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6306593418121338', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6161671280860901', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5934129953384399', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5929120182991028', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5914928913116455', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5841131806373596', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.539723813533783', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5395549535751343', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5367130041122437', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5264455080032349', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5223358273506165', 3524294, '[투데이특가] 여심저격 클루나드시계', '83', '패션시계')
('0.5090673565864563', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5037084221839905', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5032361149787903', 3544419, '[게릴라특가] 봄맞이 카페트 150x200', '71', '카페트/러그')
('0.5

## wepick_pred_a16_32_16

In [25]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a16_32_16.csv')

In [26]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6306593418121338', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6161671280860901', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5934129953384399', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5929120182991028', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5914928913116455', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5841131806373596', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.539723813533783', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5395549535751343', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5367130041122437', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5264455080032349', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5223358273506165', 3524294, '[투데이특가] 여심저격 클루나드시계', '83', '패션시계')
('0.5090673565864563', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5037084221839905', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5032361149787903', 3544419, '[게릴라특가] 봄맞이 카페트 150x200', '71', '카페트/러그')
('0.5

## wepick_pred_a4_32_16

In [40]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model_simple\prediction\wepick_pred_a4_32_16.csv')

In [41]:
print_predicted_ranks(pred_base_model['1000007'])

('0.6306593418121338', 3527569, '[투데이특가] 아디다스 그래픽스케일', '91', '반팔 티셔츠')
('0.6161671280860901', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.5934129953384399', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.5929120182991028', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.5914928913116455', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.5841131806373596', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.539723813533783', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5395549535751343', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.5367130041122437', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.5264455080032349', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.5223358273506165', 3524294, '[투데이특가] 여심저격 클루나드시계', '83', '패션시계')
('0.5090673565864563', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.5037084221839905', 3527631, '[투데이특가] 자외선차단 암막우산', '90', '생활잡화')
('0.5032361149787903', 3544419, '[게릴라특가] 봄맞이 카페트 150x200', '71', '카페트/러그')
('0.5